#### **Libraries imports:**

##### **Package installation:**

In [ ]:
# Pillow:
!pip3 install Pillow

# Exif:
!pip3 install exif

##### **Imports:**

In [ ]:
# Libraries imports:
import shutil as pysh
import pandas as pypd
import PIL
import os

from exif import Image
from PIL.ExifTags import TAGS

print('- All libraries were imported.');

#### **Setting up images:**

##### **Creating the dataset directory:**

In [ ]:
# Directory measurement variable:
dataset_existente = os.path.exists(r'/content/dataset');

# Creation of the 'dataset' directory, if it does not exist:
if not dataset_existente:
    os.system('mkdir dataset');
    print('- Dataset folder created. Populate it with images!');
else:
    # Dataset directory benchmarking:
    dataset_diretorio = os.listdir(r'/content/dataset');

    if len(dataset_diretorio) == 0:
        print('- Dataset folder already exists, necessary to populate the dataset directory with images.');
    else:
        print('- Dataset folder already exists.');

##### **Unzip of drive files:**

In [ ]:
# Parameters: .zip path, where to send it and format:
pysh.unpack_archive(r'/content/drive/MyDrive/apicultura-darknet-cnn/dataset-apicultura.zip', '/content/dataset', 'zip');

# Move files:
diretorio_original = r'/content/dataset/dataset-apicultura//';
diretorio_final = r'/content/dataset//';

for arquivo in os.listdir(r'/content/dataset/dataset-apicultura'):
    origem = diretorio_original + arquivo;
    destino = diretorio_final + arquivo;
    pysh.move(origem, destino);

# Remove original folder:
os.system('rm -rf /content/dataset/dataset-apicultura');

print('- Sucessfully unziped.');

#### **Reading images:**

##### **Creating the dataset list:**

In [ ]:
# Reading each image [in array format] manually:
dataset_directory = '/content/dataset';
dataset_array = [];

dataset = os.listdir(dataset_directory);

for aux in range(len(dataset)):
    # Captures only files with .jpg or .png extensions and increments them in the dataset_array list:
    if (dataset[aux].split('.')[1] == 'JPG' or dataset[aux].split('.')[1] == 'jpg' or dataset[aux].split('.')[1] == 'PNG' or dataset[aux].split('.')[1] == 'png'):
        dataset_array.append(dataset[aux]);

print('- Dataset list created.');

##### **Checking images:**

In [ ]:
# Checking the amount of images obtained:
print(f'- {len(dataset_array)} images were read.');

##### **Testing with one example image:**

In [ ]:
# Test:
image_path = '/content/dataset/F0010121.JPG';

with open(image_path, 'rb') as source:
    image = Image(source);

    print(f'- Source name: {source.name}');

##### **Checking image exif:**

In [ ]:
# Checking if exif exists:
image = Image(image_path);

if image.has_exif:
    info = f'has the EXIF version {image.exif_version}';
else:
    info = 'does not contain any EXIF information.';
print(f'- Image {source.name} {info}.');

##### **Exif image attributes:**

In [ ]:
# Exif attributes:
with open(image_path, 'rb') as source:
    img = Image(source);

print(f'- Exif attributes: {img.list_all()}');

##### **Image longitude:**

In [ ]:
print(f'- Longitude: {img.gps_longitude}');

In [ ]:
print(f'- Longitude reference: {img.gps_longitude_ref}.');

##### **DMS (degrees, minutes, secondss) coordinates to DD (decimal degrees) coordinates:**

In [ ]:
def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1]/60 + coords[2]/3600;

    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees;
        
    return decimal_degrees

##### **Compiling coordinates and timestamp:**

In [ ]:
# Aux variables:
image_time = [];
image_date = [];
image_latitude = [];
image_longitude = [];
path_to_image = [];
    
def image_coordinates(image_path):
    # Reading image file:
    with open(image_path, 'rb') as source:
        image = Image(source);

    if image.has_exif:
        try:
            image.gps_longitude;
            coords = (decimal_coords(image.gps_latitude, image.gps_latitude_ref), decimal_coords(image.gps_longitude, image.gps_longitude_ref));

            # Appending info:
            image_time.append(image.datetime_original[11:19]);
            image_date.append(image.datetime_original[:10]);
            image_latitude.append(coords[0]);
            image_longitude.append(coords[1]);
            path_to_image.append(image_path[17:100]);

        except AttributeError:
            print('- No coordinates');
    else:
        print(f'- {image_path} has no EXIF information.');

    print(f"- {image_path} was taken at {image.datetime_original[11:19]} on {image.datetime_original[:10]}, and has latitude: {coords[0]} and longitude {coords[1]}");

#### **Getting coordinates and timestamp for all images:**

In [ ]:
for aux in range(len(dataset_array)):
    path = f'/content/dataset/{dataset_array[aux]}';

    print(f'{aux + 1} ', end = '');

    image_coordinates(path);

#### **Getting coordinates and timestamp for all images in a dataframe:**

In [ ]:
# Calling dataframe constructor after zipping:
images_info_dataframe = pypd.DataFrame(list(zip(path_to_image, image_time, image_date, image_latitude, image_longitude)), columns =['Image', 'Time', 'Date','Latitude', 'Longitude']);

print(images_info_dataframe.head());

In [ ]:
# Writing a .xlsl file:
images_info_dataframe.to_excel('/content/images-info-lat-long.xlsx');